In [1]:
#importing libraries 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import seaborn as sns
import math
import statsmodels.api as sm

In [2]:
from sklearn.metrics import f1_score as f1
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression as logreg
from sklearn.linear_model import LinearRegression as linreg

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score as auc, accuracy_score as acc, confusion_matrix as cm 
from sklearn.metrics import precision_score as prec, recall_score as rec,average_precision_score as aps

from pandas.core.common import SettingWithCopyWarning
import warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

## random state

In [3]:
r=84

In [4]:
data = pd.read_csv('healthcare-dataset-stroke-data.csv')
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [5]:
data.shape

(5110, 12)

In [6]:
target='stroke'         
model= logreg()
lr=linreg()
scaler = MinMaxScaler()

## preprocessing

In [7]:
data=data[[z for z in data.columns.tolist() if z not in ['Unnamed: 0']]].copy()

In [8]:
data=data[[z for z in data.columns.tolist() if z not in ['ID','id']]].copy()

In [9]:
labelencode=['gender','ever_married','work_type','Residence_type','smoking_status']

for i in labelencode:
 le = LabelEncoder()
 le.fit(data[i])
 data[i]=le.transform(data[i])

In [10]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.6,1,1
1,0,61.0,0,0,1,3,0,202.21,NaN,2,1
2,1,80.0,0,1,1,2,0,105.92,32.5,2,1
3,0,49.0,0,0,1,2,1,171.23,34.4,3,1
4,0,79.0,1,0,1,3,0,174.12,24.0,2,1


In [11]:
data=(data[[z for z in data.columns.tolist() if z not in ['Unnamed: 0']]]).copy()

## missing values

In [20]:
kval_n=2

In [21]:
def kval(main,step,n=kval_n):

 klist=[]
 if (step==0):
    step=0.0001
 i=main-step*n
 if (main>=0):
  while (i<=main+step*n):
    
    klist.append(i)
    i=i+step
  return klist
 else:
  while (i>=main+step*n):
    
    klist.append(i)
    i=i+step
  return klist
  

best way to do missing value imputation is threshholding and creating an ml model where we have to predict missing value so we have to create n models if m columns have missing values and m-t columns are dropped as their missing values cross the threshhold.t-n missing values are imputed using mean mode etc.n missing values are computed by models.the threshhold for deciding t from n can be randomly selected

In [22]:
data.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [23]:
intlist=data.dtypes[data.dtypes=='int64'].index.tolist()
floatlist=data.dtypes[data.dtypes=='float64'].index.tolist()

In [25]:
flist,ilist=[[],[]]
for i in range(len(intlist)):
    if len(data[intlist[i]].value_counts())>13:
        flist.append(intlist[i])
    else :
        ilist.append(intlist[i])
floatlist.extend(flist)
intlist=ilist

### pls check wether missing values of all columns of intlist are to be imputed using classification and all other columns by regression

objcol is intlist. . .....it is actually assumed to be the columns where classification model is to be applied to impute the missing values.similarly all other columns are considered to have regression model to be applied to impute missing values

In [29]:
objcol=intlist
model_criteria=math.ceil(data.shape[0]/1500)
model_criteria

model_index=[i for i, j in enumerate(data.isnull().sum().values) if j >model_criteria] 
nonmodel_index=[i for i, j in enumerate(data.isnull().sum().values) if (j <= model_criteria)&(j>0)]
for i in nonmodel_index:
 mean=np.array(data[data[i].isnull()==False][i]).mean()
 data[i]=(data[i].fillna(mean)).copy()

In [30]:
d=data.copy()

In [31]:
missing_index1=[i for i, j in enumerate(d.isnull().sum().values) if j > 0]
missingcol1=[d.isnull().sum().index.tolist()[z] for z in missing_index1]

In [33]:
modelstore=model

In [35]:
c=0
testdata_index=[]
traindata_index=[]
for i in [data.isnull().sum().index.tolist()[z] for z in model_index]:
   testdata_index.append([])
   traindata_index.append([])
   target1=i
   missing_index=[i for i, j in enumerate(d.isnull().sum().values) if j > 0]
   missingcol=[d.isnull().sum().index.tolist()[z] for z in missing_index]
   d0=d[[z for z in d if z not in [t for t in missingcol if t not in target1]]].copy()
   testdata=d0[pd.isnull(d0[target1])==True].copy()
   testdata_index[c]=testdata.index.tolist()
   testx = testdata[[z for z in testdata.columns.tolist() if z not in [target1]]].copy()
   testy = testdata[target1].copy()
   traindata=d0[pd.isnull(d0[target1])==False].copy()
   traindata_index[c]=traindata.index.tolist()
   trainx = traindata[[z for z in traindata.columns.tolist() if z not in [target1]]].copy()

   cols=trainx.columns.tolist()
   trainx = pd.DataFrame(scaler.fit_transform(trainx),columns=cols).copy()
   testx = pd.DataFrame(scaler.fit_transform(testx),columns=cols).copy()
   
    
   trainy = traindata[target1].copy()
   c=c+1
   if (target1 in objcol):
    model=logreg
    model.fit(trainx,trainy)
    d.loc[d[d[target1].isnull()].index.tolist(),[target1]]=model.predict(testx).copy()
   else:
    model=lr
    model.fit(trainx,trainy)
    d.loc[d[d[target1].isnull()].index.tolist(),[target1]]=model.predict(testx).copy()

In [36]:
model=modelstore

In [37]:
data=d.copy()

## outliers

In [38]:
data_outlier=data.copy()

In [39]:
data_outlier_target=data.copy()

In [40]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,30.571525,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1


In [41]:
#seperating independent and dependent variables
x = (data[[z for z in data.columns.tolist() if z not in [target]]]).copy()
y =data[target].copy()

train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r,stratify=y).copy()
train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r,stratify=train_y).copy()

cols=train_x.columns.tolist()
train_x = pd.DataFrame(scaler.fit_transform(train_x),columns=cols).copy()
test_x = pd.DataFrame(scaler.fit_transform(test_x),columns=cols).copy()
val_x = pd.DataFrame(scaler.fit_transform(val_x),columns=cols).copy()

model.fit(train_x,train_y)
pred=model.predict_proba(val_x)
auc(val_y,pred[:,1])

0.8449172325976231

In [44]:
intlist=x.dtypes[x.dtypes=='int64'].index.tolist()
floatlist=x.dtypes[x.dtypes=='float64'].index.tolist()
catlist=x.dtypes[x.dtypes=='category'].index.tolist()

In [45]:
noncat=[]
for i in intlist:
    if (len(data[i].value_counts())>20):
        noncat.append(i)

In [46]:
noncat.extend(floatlist)

In [47]:
noncat

['age', 'avg_glucose_level', 'bmi']

In [48]:
outlier_list,outlierhigh,outlierlow=[[],[],[]]
for i in noncat :
    quant25 = data[i].quantile(0.25)
    quant75 = data[i].quantile(0.75)
    
    IQR = quant75 - quant25
    med = data[i].median()
    whis_low = quant25-(1.5*IQR)
    whis_high = quant75+(1.5*IQR)
    
    outlier_high = len(data[i][data[i]>(whis_high)])
    outlier_low = len(data[i][data[i]<(whis_low)])
    outlier_list.append((outlier_high+outlier_low))
    outlierhigh.append(outlier_high)
    outlierlow.append(outlier_low)
      
outlier_desc=pd.DataFrame([noncat,outlier_list,outlierhigh,outlierlow],index=['columns','No of outliers','outlier high','outlier low'],).transpose()
outlier_desc.sort_values(['No of outliers'], inplace=True ,ascending=False)
outlier_desc.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')

In [49]:
outlier_desc

,columns,No of outliers,outlier high,outlier low
0,avg_glucose_level,627,627,0
1,bmi,117,117,0
2,age,0,0,0


In [50]:
#set minimum number of outliers wrt total no of values which is to be considered to not to set to whiskers
outlier_min_whisker=0
outlier_min_whisker

0

In [51]:
outlierlist=outlier_desc[outlier_desc['No of outliers'] > outlier_min_whisker]['columns'].tolist()

In [52]:
outlierlist

['avg_glucose_level', 'bmi']

In [53]:
data=data_outlier.copy()
evallist=[]
for k in outlierlist:
 data=data_outlier.copy()
 for i in data[[z for z in noncat if z not in k]]:
     quant25 = data[i].quantile(0.25)
     quant75 = data[i].quantile(0.75)
     IQR = quant75 - quant25
     med = data[i].median()
     whis_low = quant25-(1.5*IQR)
     whis_high = quant75+(1.5*IQR)
 
     data[i][data[i]>whis_high] = whis_high+1
     data[i][data[i]<whis_low] = whis_low-1

 #seperating independent and dependent variables
 x = (data[[z for z in data.columns.tolist() if z not in [target]]]).copy()
 y =data[target].copy() 

 train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r,stratify=y).copy()
 train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r,stratify=train_y).copy()

 cols=train_x.columns.tolist()
 train_x = pd.DataFrame(scaler.fit_transform(train_x),columns=cols).copy()
 test_x = pd.DataFrame(scaler.fit_transform(test_x),columns=cols).copy()
 val_x = pd.DataFrame(scaler.fit_transform(val_x),columns=cols).copy()

 model.fit(train_x,train_y)
 pred=model.predict_proba(val_x)
 evallist.append(auc(val_y,pred[:,1]))

In [54]:
data=data_outlier_target.copy()
evallist_target=[]
for k in outlierlist:
 data=data_outlier_target.copy()
 for i in data[[z for z in noncat if z not in k]]:
     quant25 = data[i].quantile(0.25)
     quant75 = data[i].quantile(0.75)
     IQR = quant75 - quant25
     med = data[i].median()
     whis_low = quant25-(1.5*IQR)
     whis_high = quant75+(1.5*IQR)
 
     data[i][data[i]>whis_high] = whis_high+1
     data[i][data[i]<whis_low] = whis_low-1

 #seperating independent and dependent variables
 x = data[[z for z in data.columns.tolist() if z not in [target]]].copy()
 y = data[target].copy()
 x.shape, y.shape

 train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r,stratify=y).copy()
 train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r,stratify=train_y).copy()

 cols=train_x.columns.tolist()
 train_x = pd.DataFrame(scaler.fit_transform(train_x),columns=cols).copy()
 test_x = pd.DataFrame(scaler.fit_transform(test_x),columns=cols).copy()
 val_x = pd.DataFrame(scaler.fit_transform(val_x),columns=cols).copy()
    
 model.fit(train_x,train_y)
 pred=model.predict_proba(val_x)
 evallist_target.append(auc(val_y,pred[:,1]))

In [55]:
 evallist

[0.844983552631579, 0.8444132003395586]

In [56]:
 evallist_target

[0.844983552631579, 0.8444132003395586]

In [57]:
data=data_outlier_target.copy()
for i in data[[z for z in noncat if z not in k]]:
     quant25 = data[i].quantile(0.25)
     quant75 = data[i].quantile(0.75)
     IQR = quant75 - quant25
     med = data[i].median()
     whis_low = quant25-(1.5*IQR)
     whis_high = quant75+(1.5*IQR)
 
     data[i][data[i]>whis_high] = whis_high+1
     data[i][data[i]<whis_low] = whis_low-1
#seperating independent and dependent variables
x = data[[z for z in data.columns.tolist() if z not in [target]]].copy()
y = data[target].copy()
x.shape, y.shape

train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r,stratify=y).copy()
train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r,stratify=train_y).copy()

cols=train_x.columns.tolist()
train_x = pd.DataFrame(scaler.fit_transform(train_x),columns=cols).copy()
test_x = pd.DataFrame(scaler.fit_transform(test_x),columns=cols).copy()
val_x = pd.DataFrame(scaler.fit_transform(val_x),columns=cols).copy()

model.fit(train_x,train_y)
pred=model.predict_proba(val_x)
eval_target=auc(val_y,pred[:,1])

In [58]:
eval_target

0.8444132003395586

In [59]:
data=data_outlier.copy()
for i in data[[z for z in noncat if z not in k]]:
     quant25 = data[i].quantile(0.25)
     quant75 = data[i].quantile(0.75)
     IQR = quant75 - quant25
     med = data[i].median()
     whis_low = quant25-(1.5*IQR)
     whis_high = quant75+(1.5*IQR)
 
     data[i][data[i]>whis_high] = whis_high+1
     data[i][data[i]<whis_low] = whis_low-1
#seperating independent and dependent variables
x = data[[z for z in data.columns.tolist() if z not in [target]]].copy()
y = data[target].copy()
x.shape, y.shape

train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r,stratify=y).copy()
train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r,stratify=train_y).copy()

cols=train_x.columns.tolist()
train_x = pd.DataFrame(scaler.fit_transform(train_x),columns=cols).copy()
test_x = pd.DataFrame(scaler.fit_transform(test_x),columns=cols).copy()
val_x = pd.DataFrame(scaler.fit_transform(val_x),columns=cols).copy()


model.fit(train_x,train_y)
pred=model.predict_proba(val_x)
eval=auc(val_y,pred[:,1])

In [60]:
eval

0.8444132003395586

In [61]:
if (eval<eval_target):
    data=data_outlier_target.copy()
    evalvalist=[z for z in evallist_target if z>eval_target]
    outlierlist_index=[i for i, e in enumerate(evallist_target) if e in evalvalist]
    outliercol=[outlierlist[z] for z in outlierlist_index]
    print("the outliers from the target column will be removed")
else:
    data=data_outlier.copy()
    evalvalist=[z for z in evallist if z>eval]
    outlierlist_index=[i for i, e in enumerate(evallist) if e in evalvalist]
    outliercol=[outlierlist[z] for z in outlierlist_index]
    print("the outliers from the target column will not be removed")
print("the outliers from " ,outliercol, " will not be removed to minimize eval metric")
    

the outliers from the target column will not be removed
the outliers from  ['avg_glucose_level']  will not be removed to minimize eval metric


please note if multiple columns have outliers which should not be removed to minimize eval metric then process might be complex slightly

In [62]:
kp=[0.05,0.01,0.005]
kn=[-0.05,-0.01,-0.005]

In [63]:
datak=data.copy()

In [64]:
k=[-0.0005,0,0.0005]
evallistk=[]
for j in k:
 data=datak.copy()
 for i in [h for h in outlierlist if h not in outliercol]:
     quant25 = data[i].quantile(0.25)
     quant75 = data[i].quantile(0.75)
     IQR = quant75 - quant25
     med = data[i].median()
     whis_low = quant25-(1.5*IQR)
     whis_high = quant75+(1.5*IQR)
     range=np.array(data[i]).max()-np.array(data[i]).min()
     
     data[i][data[i]>whis_high] = whis_high+1-j*range
     data[i][data[i]<whis_low] = whis_low-1+j*range
 #seperating independent and dependent variables
 x = data[[z for z in data.columns.tolist() if z not in [target]]].copy()
 y = data[target].copy()
 x.shape, y.shape
    
 train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r,stratify=y).copy()
 train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r,stratify=train_y).copy()

 cols=train_x.columns.tolist()
 train_x = pd.DataFrame(scaler.fit_transform(train_x),columns=cols).copy()
 test_x = pd.DataFrame(scaler.fit_transform(test_x),columns=cols).copy()
 val_x = pd.DataFrame(scaler.fit_transform(val_x),columns=cols).copy()

 model.fit(train_x,train_y)
 pred=model.predict_proba(val_x)
 evallistk.append(auc(val_y, pred[:,1]))

In [65]:
evallistk

[0.8449968166383702, 0.844983552631579, 0.8450100806451614]

In [66]:
if (evallistk[2]==np.array(evallistk).max()):
    k=kp
elif (evallistk[0]==np.array(evallistk).max()):
    k=kn
else:
    k=[0]

### next 2 cells are for repititive whisker treatment of nontarget outlier columns

In [83]:
evallistk=[]
for j in k:
 data=datak.copy()
 for i in [h for h in outlierlist if h not in outliercol]:
     quant25 = data[i].quantile(0.25)
     quant75 = data[i].quantile(0.75)
     IQR = quant75 - quant25
     med = data[i].median()
     whis_low = quant25-(1.5*IQR)
     whis_high = quant75+(1.5*IQR)
     range=np.array(data[i]).max()-np.array(data[i]).min()
     
     data[i][data[i]>whis_high] = whis_high+1-j*range
     data[i][data[i]<whis_low] = whis_low-1+j*range
 #seperating independent and dependent variables
 x = data[[z for z in data.columns.tolist() if z not in [target]]].copy()
 y = data[target].copy()
 x.shape, y.shape
 train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r,stratify=y).copy()
 train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r,stratify=train_y).copy()

 cols=train_x.columns.tolist()
 train_x = pd.DataFrame(scaler.fit_transform(train_x),columns=cols).copy()
 test_x = pd.DataFrame(scaler.fit_transform(test_x),columns=cols).copy()
 val_x = pd.DataFrame(scaler.fit_transform(val_x),columns=cols).copy()

 model.fit(train_x,train_y)
 pred=model.predict_proba(val_x)
 evallistk.append(auc(val_y, pred[:,1]))

In [84]:
 evallistk

[0.8450100806451614,
 0.8450100806451614,
 0.8450100806451614,
 0.844983552631579,
 0.844983552631579]

#### step ke value pe dhyan do

In [85]:
kmain=k[evallistk.index(np.array(evallistk).max())]

step=kmain/(kval_n*10)

k=[z for z in kval(kmain,step,kval_n)]

In [86]:
k

[-8.1e-05, -8.55e-05, -9e-05, -9.45e-05, -9.900000000000001e-05]

keep on reducing rmse value by decreasing step whenever necassary

In [87]:
 j= k[evallistk.index(np.array(evallistk).max())]
 data=datak.copy()
 for i in [h for h in outlierlist if h not in outliercol]:
     quant25 = data[i].quantile(0.25)
     quant75 = data[i].quantile(0.75)
     IQR = quant75 - quant25
     med = data[i].median()
     whis_low = quant25-(1.5*IQR)
     whis_high = quant75+(1.5*IQR)
     range=np.array(data[i]).max()-np.array(data[i]).min()
     
     data[i][data[i]>whis_high] = whis_high+1-j*range
     data[i][data[i]<whis_low] = whis_low-1+j*range
 #seperating independent and dependent variables
 x = data[[z for z in data.columns.tolist() if z not in [target]]].copy()
 y = data[target].copy()
 x.shape, y.shape
 train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=1/4,random_state= r,stratify=y).copy()
 train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/3,random_state=r,stratify=train_y).copy()

 cols=train_x.columns.tolist()
 train_x = pd.DataFrame(scaler.fit_transform(train_x),columns=cols).copy()
 test_x = pd.DataFrame(scaler.fit_transform(test_x),columns=cols).copy()
 val_x = pd.DataFrame(scaler.fit_transform(val_x),columns=cols).copy()

 model.fit(train_x,train_y)
 pred=model.predict_proba(val_x)
 auc(val_y, pred[:,1])

0.8450100806451614

In [88]:
j

-8.1e-05

In [89]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,30.571525,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1


In [90]:
data.to_csv('FILTERED[csv].csv',index=False)

#### after running all cells from outlier heading to here go back to outlier heading and again run the cells to gain improvement in rmse as target column will have treated columns to estimate whiskers and non target column will have doubly treated target column to estimate whiskers

ideally u can keep doing this process till in non target region, the rmse does not change at all almost ......for example in some cases even decimal figures VARY ONLY SLIGHTLY

ideally u can keep doing this process till in non target region the ideal whisker is 0 i.e. slightly above 0 (0.005) or below 0( -0.005) the rmse does not change much

if target column has a huge number of outliers rmse might have massive scope of reduction by repititive process

if target column has negligible outliers then rmse might have little scope of reduction by repitive process